<a href="https://colab.research.google.com/github/ianwong74/Tweeter_Airline_Sentiment_Analysis/blob/main/csda_1040_lab2_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSDA1040 - Group 5 - Lab2 - US Airline Tweets Sentiment Analysis
** This will take up to 1 minute for initial load, Thank you for your patience*

In [1]:
# Comment these block out before publishing to Binder, packages listed in requirements.txt will be preinstalled when building the image
from IPython.utils import io

with io.capture_output() as captured: 
  !pip install html2text
  !pip install nltk
  !pip install db-sqlite3
  !pip install fastai==1.0.61
  !pip install pytest-shutil
  !pip install voila

In [2]:
import sqlite3 as db
import re
import os
import codecs
import pandas as pd
import numpy as np
import html2text
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
import shutil
import functools
import time
import warnings
import urllib.request
warnings.filterwarnings('ignore') 
np.set_printoptions(precision=6, suppress=True)
import random
import ipywidgets as widgets
from IPython.display import display

import fastai
from fastai import *
from fastai.text import *
from fastai.core import *
import pickle
from fastai.callbacks import *

In [3]:
# download pkl files from shared file from Google drive
# CSDA 1040 Team, please update the shared file id if you want to use your train result. Otherwise you are using the team's master version.
# sentiments1.pkl: https://drive.google.com/file/d/1--FqAwtWYYWglNrA-NZ4bd_wMjp27Rg_/view?usp=sharing

from IPython.utils import io
with io.capture_output() as captured: 
  !gdown --id '1--FqAwtWYYWglNrA-NZ4bd_wMjp27Rg_' --output pkl

# tweets_main.db: https://drive.google.com/file/d/1F3wvsn2eAVT964uFit8iLqZipIBowyj7/view?usp=sharing
from IPython.utils import io
with io.capture_output() as captured: 
  !gdown --id '1F3wvsn2eAVT964uFit8iLqZipIBowyj7' --output tw
cn = db.connect('tw')
cmd = cn.cursor()

In [4]:
#df = pd.read_sql('select tweet_body from tweets', cn) 
#df.head()

In [5]:
#learn = load_learner('.','pkl')
#ts = pd.read_sql('select tweet_body from tweets', cn)
#n = random.randint(0,len(ts)-1)
#t = ts.loc[n]
#print(t)
#p = learn.predict(t)
#print(p)

In [6]:
# define class & properties
class Sentiments():
    # class init function, load model jester_jokes (jj) from exported model from shared Google drive prepared by csda_1040_lab1.ipynb
    def __init__(self):
      # load mode to learner
      self.__learn = load_learner('.','pkl')
      # load tweets_main.db db-sqlite3 file, extract tweet_body (text)
      self.__tweets = pd.read_sql('select tweet_body from tweets', cn)
      n = random.randint(0,len(self.__tweets)-1)
      self.__get_preds(n)
 
    def __get_preds(self, n):
      self.__tweet_body = self.__tweets.loc[n]
      pred,_,_ = self.__learn.predict(self.__tweet_body)
      thumbs_up = '\U0001F44D'
      thumbs_down = '\U0001F44E'
      if str(pred) == '1':
        pred_rslt = thumbs_up
      else: 
        pred_rslt = thumbs_down
      self.__preds = pred_rslt
      
    def __next_preds(self):
      n = random.randint(0,len(self.__tweets)-1)
      self.__get_preds(n)



In [7]:
#sentiments = Sentiments()
#sentiments._Sentiments__tweets.head()
#print(sentiments._Sentiments__preds)
#print(sentiments._Sentiments__tweet_body.values)
#x = sentiments._Sentiments__tweet_body.values[0]
#print(np.fromstring(x, dtype=str))
#x

In [8]:
#sentiments._Sentiments__next_preds()
#print(sentiments._Sentiments__preds)
#print(sentiments._Sentiments__tweet_body.values)

In [9]:
# create the recommender
# from io import StringIO
sentiments = Sentiments()
ui_html = None
ui_slider = None
ui_pred = None
ui_tweet_body = None
ui_sumbit = None
ui_container = None


def ftn_render_cold_start():
  # renders the cold-start UI, load first tweet and sentiment
  global ui_cold_start_container
  global ui_sentiment_lbl
  global ui_tweet_body_txt
  global ui_sumbit
  pred = sentiments._Sentiments__preds
  txt = sentiments._Sentiments__tweet_body.values[0]
  ui_tweet_body_txt = widgets.Text(layout=widgets.Layout(width="70%"))
  ui_tweet_body_txt.value = txt
  ui_sentiment_lbl = widgets.Label('Predicted Sentiment: ' + pred)
  ui_sumbit = widgets.Button(description="Next")
  ui_sumbit.layout.margin = '20px'
  ui_sumbit.on_click(ftn_on_click)
  items = [ui_tweet_body_txt, ui_sentiment_lbl, ui_sumbit]
  box = widgets.VBox(items)
  box.layout = widgets.Layout(align_items='center', width='auto')
  display(box)
  ui_cold_start_container = box

def ftn_on_click(self):
  global ui_cold_start_container
  global ui_sentiment_lbl
  global ui_tweet_body_txt
  global ui_sumbit
  sentiments._Sentiments__next_preds()
  pred = sentiments._Sentiments__preds
  txt = sentiments._Sentiments__tweet_body.values[0]
  ui_tweet_body_txt = widgets.Text(layout=widgets.Layout(width="70%"))
  ui_tweet_body_txt.value = txt
  ui_sentiment_lbl = widgets.Label('Predicted Sentiment: ' + pred)
  items = [ui_tweet_body_txt, ui_sentiment_lbl, ui_sumbit]
  box = widgets.VBox(items)
  box.layout = widgets.Layout(align_items='center', width='auto')
  display(box)
  ui_cold_start_container = box


In [10]:
ftn_render_cold_start()